### Non-ideal reaction-diffusion systems

In the last notebook you learned the basics of phase separation driven by the competition between entropy (favoring mixing) and non-ideal interactions (favoring demixing). In this notebook you will see how first order reactions can introduce patterns of a certain length scale.

In [2]:
''' Import the packages that we need for this tutorial '''
import sys          # sys allows us to add paths to packages
import numpy as np  # numpy 
import matplotlib.pyplot as plt

#sys.path.append('../..')  # add the pde package to the python path
sys.path.append('/Users/jankirschbaum/Documents/GitHub/D2/py-pde')
import pde

In [3]:
# define the partial differential equation
k = 1.e-2
c0 = 0.2
eq = pde.PDE({'c': f'laplace(2 * c * (1 - c) * (1 - 2*c) - laplace(c)) + {k} * (c - {c0})'})
eq.expressions

{'c': '0.01*c + laplace(2*c*(c - 1)*(2*c - 1) - laplace(c)) - 0.002'}